In [1]:
import numpy as np
import pandas as pd 
import requests
import json
from bs4 import BeautifulSoup, SoupStrainer

In [1348]:
import importlib

In [2]:
from crawler.overview_data import get_overview_data

In [3]:
%time y = get_overview_data(from_file=True)

Wall time: 33.4 s


In [923]:
%time overview_htmls = get_overview_htmls(from_file=True)

Wall time: 7.3 s


In [1298]:
test_IDs = player_overview_data['ID'].sample(50)
player_htmls = get_player_htmls(test_IDs)

In [1320]:
x = parse_player_detailed_data(player_htmls, constants)

In [528]:
def main(from_file=False):
    player_overview_data = get_overview_data(from_file)
    IDs = player_overview_data['ID']
    player_detailed_data = get_player_detailed_data(IDs, from_file)
    merged = player_overview_data.merge(player_detailed_data, on='ID')
    return merged